In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import kinematics
from util import SE3
import trimesh
import trimesh.viewer.notebook
import time
from IPython.display import clear_output

# Robot Definition
The robot can be seen in the following image:

![Arm image](arm_img.png){width=30%}

It has 5 joints in a 2DoF shoulder, 1DoF elbow, 2DoF wrist configuration (the 2 wrist joints are both perpendicular to the elbow, while the elbow is parallel with the second shoulder joint).

Although we define the robot parameters in kinematics.py, we'll re-defined them here for reference.

In [ ]:
# Constants
BRUSH_LENGTH = 0.115 + .1
HAND_LENGTH = .031
FOREARM_LENGTH = .149
UPPERARM_LENGTH = .148
ELBOW_LATERAL_OFFSET = 0.03205
BASE_TO_SHOULDER = 0.06525

Using Lynch & Park notations, we can define the rest transform & screw axis as:

In [ ]:
# Robot Definition
REST_TRANSFORMS = [
    SE3.P([0, 0, BASE_TO_SHOULDER]),
    SE3.P([0, 0, UPPERARM_LENGTH]),
    SE3.P([ELBOW_LATERAL_OFFSET, 0, FOREARM_LENGTH]),
    SE3.P([0, 0, HAND_LENGTH]),
    SE3.P([0, 0, BRUSH_LENGTH]),
]
SCREW_AXES = np.array([
    [0, 0, 1, 0, 0, 0],
    [1, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [1, 0, 0, 0, 0, 0],
])

In [ ]:
# Unit tests
!python -m unittest test_kinematics.py

# FK

In [ ]:
T, joint_Ts = kinematics.fk([0, 1, 0, 0, 0])
print('Brush Tip Pose:')
display(T)
print('Joint poses:')
for pose in joint_Ts:
    print('\t', pose[:, 3])

# Visualize
kinematics.visualize_robot(joint_Ts).show()

In [ ]:
# "Animation" (sucks)
if False:
    for i, angle in enumerate(np.linspace(0, 2, 10)):
        T, joint_Ts = kinematics.fk([0, angle, 0, 0, 0])
        # with open(f'/tmp/{i:03d}.png', 'wb') as f:
        #     f.write(kinematics.visualize_robot(joint_Ts).save_image())
        display(kinematics.visualize_robot(joint_Ts).show())
        time.sleep(0.3)
        clear_output(wait=True)

# IK

In [ ]:
# Let's do a practice IK solve, using a random joint configuration to create our goal pose.
# We can also compare the results when we try to fit full SE3 vs when we ignore the spin axis.

np.random.seed(8675309)
thetas_init = np.random.uniform(-np.pi / 2, np.pi / 2, size=(5,))
print('IK ground truth:', thetas_init)
T_goal, T_links = kinematics.fk(thetas_init)
meshes1 = kinematics.robot_meshes(T_links, color=(0, 255, 0, 255))
for mesh in meshes1:
    mesh.apply_transform(SE3.P([0, 0, -0.001])) # So we can see the goal pose

success, ik_result = kinematics.ik(T_goal, ignore_spin_axis=False)
print('IK result 1:    ', ik_result)
meshes2 = kinematics.robot_meshes(kinematics.fk(ik_result)[1], color=(0, 0, 255, 255))

success, ik_result = kinematics.ik(T_goal, ignore_spin_axis=True)
print('IK result 2:    ', ik_result)
meshes3 = kinematics.robot_meshes(kinematics.fk(ik_result)[1], color=(255, 0, 0, 255))


goal_mesh = trimesh.creation.axis(axis_length=0.3, origin_size=0.01).apply_transform(T_goal)
print(T_goal)

kinematics.create_scene(meshes1 + meshes2 + meshes3 + [goal_mesh]).show()

In [ ]:
# Now let's try to solve IK for some useful poses
def solve_and_mesh(T_goal, color=None, **ik_kwargs):
    success, ik_result = kinematics.ik(T_goal, **ik_kwargs)
    print(ik_result.round(3))
    assert success, f'IK failed: {ik_result}'

    goal_pose = trimesh.creation.axis(axis_length=0.3, origin_size=0.01).apply_transform(T_goal)
    return kinematics.robot_meshes(kinematics.fk(ik_result)[1], color=color) + [goal_pose]


kwargs = {'ignore_spin_axis': True, 'elbow_mode': 'pos'}
pose_glass_straight = SE3.P([0, 0.3, 0.35]) @ SE3.RotX(-np.pi / 2)
pose_paint_dip = SE3.P([0, -0.2, 0]) @ SE3.RotX(-np.pi)

meshes = []
for pose in [pose_glass_straight, pose_paint_dip]:
    meshes += solve_and_mesh(pose, **kwargs)

kinematics.create_scene(meshes).show()

In [ ]:
# Finally, let's try a painting trajectory
def solve_and_mesh_2(T_goal, color=None, **ik_kwargs):
    success, ik_result = kinematics.ik(T_goal, **ik_kwargs)
    assert success, f'IK failed: {ik_result}'
    goal = trimesh.primitives.Sphere(radius=0.01, transform=T_goal)
    goal.visual.face_colors = color
    return kinematics.robot_meshes(kinematics.fk(ik_result)[1], color=color) + [goal]

glass_center = SE3.P([0, 0.3, 0.33]) @ SE3.RotX(-np.pi / 2)

# t = np.linspace(0, 2 * np.pi, 10)[:, None]
# art_traj = np.hstack((np.cos(t), t * 0, np.sin(t))) * 0.05
# colors = np.hstack((220 - 30 * t, 31 + 30 * t, t*0, 255 * np.ones_like(t)))
t = np.linspace(-1, 1, 10)[:, None]
art_traj = np.hstack((0.17 * t, t * 0, t * 0))
colors = np.hstack((128 - 100 * t, 128 + 100 * t, t*0, 255 * np.ones_like(t)))

poses = [SE3.P(point) @ glass_center for point in art_traj]

kinematics.create_scene(
    sum([solve_and_mesh_2(pose, color=c, **kwargs) for pose, c in zip(poses, colors)], [])).show()


# IK (old)

In [ ]:
# IK
def ik_elbow(x, z, elbow_in=True):
    # Uses law of cosines to find shoulder and elbow angle, given planar x/z
    a, b, c = UPPERARM_LENGTH, FOREARM_LENGTH, np.linalg.norm([x, z])
    gamma = np.arccos((a**2 + b**2 - c**2) / (2 * a * b))
    beta = np.arccos((a**2 + c**2 - b**2) / (2 * a * c))
    triangle_angle = np.arctan2(z, x)
    if elbow_in:
        shoulder_angle_from_horiz = triangle_angle + beta
        elbow_angle = -np.pi + gamma
    else:
        shoulder_angle_from_horiz = triangle_angle - beta
        elbow_angle = np.pi - gamma
    return np.rad2deg(shoulder_angle_from_horiz - np.pi/2), np.rad2deg(elbow_angle)

def ik_perpendicular(base_P_tip, elbow_in=True):  # position of brush tip relative to base frame
    angles = [0, 0, 0, 0, 0]
    base_P_wrist = base_P_tip - np.array([0, HAND_LENGTH, 0])
    shoulder_P_wrist_3d = base_P_wrist - BASE_TO_SHOULDER
    shoulder_P_wrist_2d = np.array([np.linalg.norm(shoulder_P_wrist_3d[:2]), shoulder_P_wrist_3d[2]])
    angles[1], angles[2] = ik_elbow(shoulder_P_wrist_2d[0], shoulder_P_wrist_2d[1], elbow_in=elbow_in)
    angles[0] = np.rad2deg(np.arctan2(shoulder_P_wrist_3d[1], shoulder_P_wrist_3d[0])) - 90
    angles[3] = 0  # TODO: this depends on base angle in a weird way
    angles[4] = 90 # TODO: this depends on base angle in a weird way
    return angles

In [ ]:
# Unit test
# TODO: fix this code... ???
base_P_tip = np.array([UPPERARM_LENGTH, HAND_LENGTH, FOREARM_LENGTH]) + BASE_TO_SHOULDER
expected_angles = [-90, -90, 90, -90, 90]
actual_angles = ik_perpendicular(base_P_tip, elbow_in=False)
np.testing.assert_allclose(actual_angles, expected_angles, err_msg='IK unit test failed')
expected_angles = [-90, 0, -90, -90, 90]
actual_angles = ik_perpendicular(base_P_tip, elbow_in=True)
np.testing.assert_allclose(actual_angles, expected_angles, err_msg='IK unit test failed')